## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Cabo San Lucas,MX,86.00,overcast clouds,22.89,-109.91,Hotel Tesoro Los Cabos
1,1,Kodiak,US,60.80,clear sky,57.79,-152.41,Best Western Kodiak Inn and Convention Center
2,2,Puerto Ayora,EC,66.99,overcast clouds,-0.74,-90.35,Finch Bay Galapagos Hotel
3,3,Belmonte,BR,75.20,scattered clouds,-15.86,-38.88,Pousada Monte Carmelo O Pão
4,4,Cururupu,BR,75.42,clear sky,-1.83,-44.87,Pousada Través'cia


In [3]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [4]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [5]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[(vacation_df['City'] == 'Pacifica')]
vacation_end = vacation_df.loc[(vacation_df['City'] == 'Pacifica')]
vacation_stop1 = vacation_df.loc[(vacation_df['City'] ==  'Portland')]
vacation_stop2 = vacation_df.loc[(vacation_df['City'] == 'Gillette')]
vacation_stop3 = vacation_df.loc[(vacation_df['City'] == 'Carlsbad')] 

In [6]:
vacation_end

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
65,70,Pacifica,US,80.6,clear sky,37.61,-122.49,"Pacifica Lighthouse, Trademark Collection by W..."


In [7]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.

vs = (vacation_start.to_numpy())
start = tuple(vs[0][5:7])
ve = (vacation_end.to_numpy())
end = tuple(ve[0][5:7])
s1 = (vacation_stop1.to_numpy())
stop1 = tuple(s1[0][5:7])
s2 = (vacation_stop2.to_numpy())
stop2 = tuple(s2[0][5:7])
s3 = (vacation_stop3.to_numpy())
stop3 = tuple(s3[0][5:7])

In [8]:
type(start)

tuple

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

travel_mode = {'driving', 'bicycling', 'walking'}
waypoints = (f"{stop1[0]}, {stop1[1]}",f"{stop2[0]}, {stop2[1]}", f"{stop3[0]}, {stop3[1]}") 
#start = f"{start[0]}, {start[1]}"
#end = f"{end[0]}, {end[1]}"
# 7a Set parameters.
params = {
    'mode' : travel_mode,
    'origin' : start,
    'destination' : end,
    'waypoints' : waypoints,
    'key': g_key
}
params
# Base URL
base_url = 'https://maps.googleapis.com/maps/api/directions/json'

# Make request.
route = requests.get(base_url, params=params).json()

#route
fig = gmaps.figure()
my_route = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')
fig.add_layer(my_route)
fig



Figure(layout=FigureLayout(height='420px'))

In [10]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3], ignore_index=True)
print(itinerary_df)


   City_ID      City Country  Max Temp Current Description    Lat     Lng  \
0       70  Pacifica      US      80.6           clear sky  37.61 -122.49   
1      575  Portland      US      86.0          few clouds  45.52 -122.68   
2      374  Gillette      US      91.4           clear sky  44.29 -105.50   
3      586  Carlsbad      US      80.6           clear sky  33.16 -117.35   

                                          Hotel Name  
0  Pacifica Lighthouse, Trademark Collection by W...  
1                      Portland Marriott City Center  
2                       National 9 Inn - Gillette WY  
3                          Carlsbad Inn Beach Resort  


In [11]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
# 4a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp =  itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))